https://geopandas.org/en/stable/gallery/polygon_plotting_with_folium.html

`python3 -m pip install geopandas`

Successfully installed fiona-1.8.20 geopandas-0.10.2 munch-2.5.0 shapely-1.8.0

In [ ]:
import geopandas as gpd
import pandas as pd
import folium
import matplotlib.pyplot as plt
import pymongo
import json

In [ ]:
mdb_uri = 'localhost'
client = pymongo.MongoClient(mdb_uri)

mdb_db='osm'

mdb_col='sweden.landuse'
filter={'properties.fclass': 'forest',
  'geometry': {
     '$near': {
       '$geometry': {
          'type': 'Point' ,
          'coordinates': [ 17.58, 59.48 ]
       },
       '$maxDistance': 1000,
       '$minDistance': 0
     }
   }
}
retrieve=5
df = gpd.GeoDataFrame.from_features(client[mdb_db][mdb_col].find(filter).limit(retrieve))

df.head()

In [ ]:
df.plot(figsize=(6, 6))
plt.show()

In [ ]:
m = folium.Map(location=[59.48,17.58], zoom_start=14, tiles='openstreetmap')
for _, r in df.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(r['name']).add_to(geo_j)
    geo_j.add_to(m)
m

In [ ]:
df['centroid'] = df.centroid

In [ ]:
df.head()

In [ ]:
for _, r in df.iterrows():
    lat = r['centroid'].y
    lon = r['centroid'].x
    folium.Marker(location=[lat, lon],
                  popup='Name:&nbsp;<b>{}</b> <br> Class:&nbsp;<a href="https://wiki.openstreetmap.org/wiki/Tag:landuse={}" target="_blank">{}</a> <br> <i>Code:&nbsp;{}</i>'.format(r['name'], r['fclass'], r['fclass'], r['code'])).add_to(m)

m

In [ ]:
mdb_col='sweden.roads'
filter={'geometry': {
     '$near': {
       '$geometry': {
          'type': 'Point' ,
          'coordinates': [ 17.58, 59.48 ]
       },
       '$maxDistance': 1000,
       '$minDistance': 0
     }
   }
}
retrieve=100
df = gpd.GeoDataFrame.from_features(client[mdb_db][mdb_col].find(filter).limit(retrieve))

for _, r in df.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])#.simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'color': 'green', 'weight':' 4'})
    folium.Popup(r['name']).add_to(geo_j)
    geo_j.add_to(m)
m